In [1]:
from qiskit import *
import numpy as np
import matplotlib.pyplot as plt
from cmath import exp,sin,cos
from qiskit.ignis.mitigation.measurement import (CompleteMeasFitter,complete_meas_cal)
from qiskit import IBMQ


/tmp/ipykernel_12187/2495068230.py:5: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import (CompleteMeasFitter,complete_meas_cal)


In [2]:
IBMQ.load_account()

/tmp/ipykernel_12187/485379158.py:1: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.load_account()
/tmp/ipykernel_12187/485379158.py:1: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.load_account()


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')


In [4]:
qcomp = provider.get_backend('ibmq_manila')


In [ ]:
qcomp.status().to_dict()




In [6]:
O12=0.58381263
O13=0.1504474
O23=0.7347836
cp_phi=0

In [7]:
mq21=7.5*pow(10,-5)t
mq31=2.5*pow(10,-3)
E=1
peq=[]
pmq=[]
puq=[]

In [ ]:
for t in range (0,1200,100):
    circ=QuantumCircuit(2,2)
    circ.initialize('10',circ.qubits)
    circ.cx(1,0)
    circ.cu(2*O23,0,0,0,0,1)
    circ.cx(1,0)
    circ.x(0)
    circ.cu(2*O13,cp_phi,-cp_phi,0,0,1)
    circ.x(0)
    circ.x(1)
    circ.cu(2*O12,0,0,0,1,0)
    circ.x(1)
    circ.p(-(2*mq21*1.27*t)/(E),0)
    circ.p(-(2*mq31*1.27*t)/(E),1)
    circ.x(1)
    circ.cu(-2*O12,0,0,0,1,0)
    circ.x(1)
    circ.x(0)
    circ.cu(-2*O13,cp_phi,-cp_phi,0,0,1)
    circ.x(0)
    circ.cx(1,0)
    circ.cu(-2*O23,0,0,0,0,1)
    circ.cx(1,0)
    circ.measure([0,1],[0,1])
    job=execute(circ,backend=qcomp)
    device_results=job.result().get_counts(circ)
    cal_circuits,state_labels=complete_meas_cal(qr=circ.qregs[0],circlabel='measerrormitigationcal')
    cal_job=execute(cal_circuits,backend=qcomp,optimization_level=0)
    cal_results=cal_job.result()
    meas_fitter=CompleteMeasFitter(cal_results,state_labels)
    meas_filter=meas_fitter.filter
    mitigated_results=meas_filter.apply(device_results)
    total = sum(mitigated_results.values())
    result = {key: value / total for key, value in mitigated_results.items()}
    try:
        peq.append(result['00'])
    except:
        peq.append(0)
    try:
        pmq.append(result['01'])
    except:
        pmq.append(0)
    try:
        puq.append(result['10'])
    except:
        puq.append(0)
